<a href="https://colab.research.google.com/github/brendontj/tcc-lol-stats/blob/master/LOL_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import math
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder   

In [83]:
df = pd.read_csv('Result_1.csv')
print(df.shape)

(69300, 60)


#Utils functions



In [84]:
def get_dragon_names(arr):
  dragons = []
  for i in arr:
    if not pd.isna(i):
      for j in i[1:-1].split(','):
        if j not in dragons:
          dragons.append(j)
  return dragons

def number_of_dragons(arr):
  if arr == 0:
    return 0
  return len(arr[1:-1].split(','))

#Model to predict winner at 05 minutes of game

###Transform dragons data

In [85]:
df_1 = df.copy()

df_1['red_team_dragons'] = df_1['red_team_dragons'].fillna(0)
df_1['blue_team_dragons'] = df_1['blue_team_dragons'].fillna(0)
print(df_1.shape)
df_1['len_red_team_dragons'] = df_1['red_team_dragons'].apply(lambda x: number_of_dragons(x))
df_1['len_blue_team_dragons'] = df_1['blue_team_dragons'].apply(lambda x: number_of_dragons(x))
print(df_1.shape)

(69300, 60)
(69300, 62)


##Filter just MD1, because MD2,MD3 and MD5 are not labeled
### TODO - resolver possivel problema na extracao dos jogos md2/md3/md5

In [87]:
df_f = df_1.loc[df_1['best_of'] == 1]
print(df_f.shape)

(44380, 62)


In [88]:
name_of_columns = [
                                     'game_external_ref',
                                     'patch_version',
                                     'blue_team_ref',
                                     'red_team_ref',
                                     'blue_team_total_gold',
                                     'blue_team_inhibitors',
                                     'blue_team_towers',
                                     'blue_team_barons',
                                     'blue_team_total_kills',	
                                     'red_team_total_gold',
                                     'red_team_inhibitors',	
                                     'red_team_towers',	
                                     'red_team_barons',	
                                     'red_team_total_kills',
                                     'game_timestamp',
                                     'game_number',
                                     'team_a_side',
                                     'team_b_side',
                                     'team_a_record_wins',                                  
                                     'team_b_record_wins',
                                     'team_a_record_losses',
                                     'team_b_record_losses',                                                                     
                                     'best_of',
                                     'red_team_dragons',
                                     'blue_team_dragons',
                                     'participant_id_player_1',
                                     'champion_name_player_1',
                                     'role_player_1',
                                     'level_player_1',
                                     'kills_player_1',
                                     'deaths_player_1',
                                     'assists_player_1',
                                     'total_gold_earned_player_1',
                                     'creep_score_player_1',
                                     'kill_participation_player_1',
                                     'champion_damage_share_player_1',
                                     'wards_placed_player_1',
                                     'wards_destroyed_player_1',
                                     'participant_id_player_2',
                                     'champion_name_player_2',
                                     'role_player_2',
                                     'level_player_2',
                                     'kills_player_2',
                                     'deaths_player_2',
                                     'assists_player_2',
                                     'total_gold_earned_player_2',
                                     'creep_score_player_2',
                                     'kill_participation_player_2',
                                     'champion_damage_share_player_2',
                                     'wards_placed_player_2',
                                     'wards_destroyed_player_2',
                                     'participant_id_player_3',
                                     'champion_name_player_3',
                                     'role_player_3',
                                     'level_player_3',
                                     'kills_player_3',
                                     'deaths_player_3',
                                     'assists_player_3',
                                     'total_gold_earned_player_3',
                                     'creep_score_player_3',
                                     'kill_participation_player_3',
                                     'champion_damage_share_player_3',
                                     'wards_placed_player_3',
                                     'wards_destroyed_player_3',
                                     'participant_id_player_4',
                                     'champion_name_player_4',
                                     'role_player_4',
                                     'level_player_4',
                                     'kills_player_4',
                                     'deaths_player_4',
                                     'assists_player_4',
                                     'total_gold_earned_player_4',
                                     'creep_score_player_4',
                                     'kill_participation_player_4',
                                     'champion_damage_share_player_4',
                                     'wards_placed_player_4',
                                     'wards_destroyed_player_4',
                                     'participant_id_player_5',
                                     'champion_name_player_5',
                                     'role_player_5',
                                     'level_player_5',
                                     'kills_player_5',
                                     'deaths_player_5',
                                     'assists_player_5',
                                     'total_gold_earned_player_5',
                                     'creep_score_player_5',
                                     'kill_participation_player_5',
                                     'champion_damage_share_player_5',
                                     'wards_placed_player_5',
                                     'wards_destroyed_player_5',
                                     'participant_id_player_6',
                                     'champion_name_player_6',
                                     'role_player_6',
                                     'level_player_6',
                                     'kills_player_6',
                                     'deaths_player_6',
                                     'assists_player_6',
                                     'total_gold_earned_player_6',
                                     'creep_score_player_6',
                                     'kill_participation_player_6',
                                     'champion_damage_share_player_6',
                                     'wards_placed_player_6',
                                     'wards_destroyed_player_6',
                                     'participant_id_player_7',
                                     'champion_name_player_7',
                                     'role_player_7',
                                     'level_player_7',
                                     'kills_player_7',
                                     'deaths_player_7',
                                     'assists_player_7',
                                     'total_gold_earned_player_7',
                                     'creep_score_player_7',
                                     'kill_participation_player_7',
                                     'champion_damage_share_player_7',
                                     'wards_placed_player_7',
                                     'wards_destroyed_player_7',                                
                                     'participant_id_player_8',
                                     'champion_name_player_8',
                                     'role_player_8',
                                     'level_player_8',
                                     'kills_player_8',
                                     'deaths_player_8',
                                     'assists_player_8',
                                     'total_gold_earned_player_8',
                                     'creep_score_player_8',
                                     'kill_participation_player_8',
                                     'champion_damage_share_player_8',
                                     'wards_placed_player_8',
                                     'wards_destroyed_player_8',                                     
                                     'participant_id_player_9',
                                     'champion_name_player_9',
                                     'role_player_9',
                                     'level_player_9',
                                     'kills_player_9',
                                     'deaths_player_9',
                                     'assists_player_9',
                                     'total_gold_earned_player_9',
                                     'creep_score_player_9',
                                     'kill_participation_player_9',
                                     'champion_damage_share_player_9',
                                     'wards_placed_player_9',
                                     'wards_destroyed_player_9',                                
                                     'participant_id_player_10',
                                     'champion_name_player_10',
                                     'role_player_10',
                                     'level_player_10',
                                     'kills_player_10',
                                     'deaths_player_10',
                                     'assists_player_10',
                                     'total_gold_earned_player_10',
                                     'creep_score_player_10',
                                     'kill_participation_player_10',
                                     'champion_damage_share_player_10',
                                     'wards_placed_player_10',
                                     'wards_destroyed_player_10',
                                     'game_winner' # Game winner; 0 if team_a win the game and 1 if team_b win the game
                                     ]
df_5_minutes = pd.DataFrame(columns=name_of_columns)

In [93]:
games_ref_list = df_f['game_external_ref'].unique()

for i in range(len(games_ref_list)):
    sub_df = df_f.loc[df_f['game_external_ref'] == games_ref_list[i]]
    
    game_timestamp_list = sub_df['game_timestamp'].unique()
    
    micro_df = sub_df.loc[sub_df['game_timestamp'] == game_timestamp_list[1]]
    
    if int(micro_df.iloc[[0]].team_a_game_wins.values[0]) == 1 and int(micro_df.iloc[[0]].team_b_game_wins.values[0]) == 0:
      game_winner = 0
    elif int(micro_df.iloc[[0]].team_a_game_wins.values[0]) == 0 and int(micro_df.iloc[[0]].team_b_game_wins.values[0]) == 1:
      game_winner = 1

    data = [(micro_df.iloc[[0]].game_external_ref).values[0],
            (micro_df.iloc[[0]].patch_version).values[0],
            (micro_df.iloc[[0]].blue_team_ref).values[0],
            (micro_df.iloc[[0]].red_team_ref).values[0],        
            (micro_df.iloc[[0]].blue_team_total_gold).values[0],
            (micro_df.iloc[[0]].blue_team_inhibitors).values[0],
            (micro_df.iloc[[0]].blue_team_towers).values[0],
            (micro_df.iloc[[0]].blue_team_barons).values[0],
            (micro_df.iloc[[0]].blue_team_total_kills).values[0],
            (micro_df.iloc[[0]].red_team_total_gold).values[0],
            (micro_df.iloc[[0]].red_team_inhibitors).values[0],
            (micro_df.iloc[[0]].red_team_towers).values[0],          
            (micro_df.iloc[[0]].red_team_barons).values[0],
            (micro_df.iloc[[0]].red_team_total_kills).values[0],
            (micro_df.iloc[[0]].game_timestamp).values[0],
            (micro_df.iloc[[0]].game_number).values[0],
            (micro_df.iloc[[0]].team_a_side).values[0],
            (micro_df.iloc[[0]].team_b_side).values[0],
            (micro_df.iloc[[0]].team_a_record_wins).values[0],            
            (micro_df.iloc[[0]].team_b_record_wins).values[0],
            (micro_df.iloc[[0]].team_a_record_losses).values[0],
            (micro_df.iloc[[0]].team_b_record_losses).values[0],
            (micro_df.iloc[[0]].best_of).values[0],
            (micro_df.iloc[[0]].len_red_team_dragons).values[0],
            (micro_df.iloc[[0]].len_blue_team_dragons).values[0],
            (micro_df.iloc[[0]].participant_id).values[0],
            (micro_df.iloc[[0]].champion_name).values[0],
            (micro_df.iloc[[0]].role).values[0],
            (micro_df.iloc[[0]].level).values[0],
            (micro_df.iloc[[0]].kills).values[0],
            (micro_df.iloc[[0]].deaths).values[0],
            (micro_df.iloc[[0]].assists).values[0],
            (micro_df.iloc[[0]].total_gold_earned).values[0],
            (micro_df.iloc[[0]].creep_score).values[0],
            (micro_df.iloc[[0]].kill_participation).values[0],
            (micro_df.iloc[[0]].champion_damage_share).values[0],
            (micro_df.iloc[[0]].wards_placed).values[0],
            (micro_df.iloc[[0]].wards_destroyed).values[0],
            (micro_df.iloc[[1]].participant_id).values[0],
            (micro_df.iloc[[1]].champion_name).values[0],
            (micro_df.iloc[[1]].role).values[0],
            (micro_df.iloc[[1]].level).values[0],
            (micro_df.iloc[[1]].kills).values[0],
            (micro_df.iloc[[1]].deaths).values[0],
            (micro_df.iloc[[1]].assists).values[0],
            (micro_df.iloc[[1]].total_gold_earned).values[0],
            (micro_df.iloc[[1]].creep_score).values[0],
            (micro_df.iloc[[1]].kill_participation).values[0],
            (micro_df.iloc[[1]].champion_damage_share).values[0],
            (micro_df.iloc[[1]].wards_placed).values[0],
            (micro_df.iloc[[1]].wards_destroyed).values[0],           
            (micro_df.iloc[[2]].participant_id).values[0],
            (micro_df.iloc[[2]].champion_name).values[0],
            (micro_df.iloc[[2]].role).values[0],
            (micro_df.iloc[[2]].level).values[0],
            (micro_df.iloc[[2]].kills).values[0],
            (micro_df.iloc[[2]].deaths).values[0],
            (micro_df.iloc[[2]].assists).values[0],
            (micro_df.iloc[[2]].total_gold_earned).values[0],
            (micro_df.iloc[[2]].creep_score).values[0],
            (micro_df.iloc[[2]].kill_participation).values[0],
            (micro_df.iloc[[2]].champion_damage_share).values[0],
            (micro_df.iloc[[2]].wards_placed).values[0],
            (micro_df.iloc[[2]].wards_destroyed).values[0],           
            (micro_df.iloc[[3]].participant_id).values[0],
            (micro_df.iloc[[3]].champion_name).values[0],
            (micro_df.iloc[[3]].role).values[0],
            (micro_df.iloc[[3]].level).values[0],
            (micro_df.iloc[[3]].kills).values[0],
            (micro_df.iloc[[3]].deaths).values[0],
            (micro_df.iloc[[3]].assists).values[0],
            (micro_df.iloc[[3]].total_gold_earned).values[0],
            (micro_df.iloc[[3]].creep_score).values[0],
            (micro_df.iloc[[3]].kill_participation).values[0],
            (micro_df.iloc[[3]].champion_damage_share).values[0],
            (micro_df.iloc[[3]].wards_placed).values[0],
            (micro_df.iloc[[3]].wards_destroyed).values[0],           
            (micro_df.iloc[[4]].participant_id).values[0],
            (micro_df.iloc[[4]].champion_name).values[0],
            (micro_df.iloc[[4]].role).values[0],
            (micro_df.iloc[[4]].level).values[0],
            (micro_df.iloc[[4]].kills).values[0],
            (micro_df.iloc[[4]].deaths).values[0],
            (micro_df.iloc[[4]].assists).values[0],
            (micro_df.iloc[[4]].total_gold_earned).values[0],
            (micro_df.iloc[[4]].creep_score).values[0],
            (micro_df.iloc[[4]].kill_participation).values[0],
            (micro_df.iloc[[4]].champion_damage_share).values[0],
            (micro_df.iloc[[4]].wards_placed).values[0],
            (micro_df.iloc[[4]].wards_destroyed).values[0],          
            (micro_df.iloc[[5]].participant_id).values[0],
            (micro_df.iloc[[5]].champion_name).values[0],
            (micro_df.iloc[[5]].role).values[0],
            (micro_df.iloc[[5]].level).values[0],
            (micro_df.iloc[[5]].kills).values[0],
            (micro_df.iloc[[5]].deaths).values[0],
            (micro_df.iloc[[5]].assists).values[0],
            (micro_df.iloc[[5]].total_gold_earned).values[0],
            (micro_df.iloc[[5]].creep_score).values[0],
            (micro_df.iloc[[5]].kill_participation).values[0],
            (micro_df.iloc[[5]].champion_damage_share).values[0],
            (micro_df.iloc[[5]].wards_placed).values[0],
            (micro_df.iloc[[5]].wards_destroyed).values[0],           
            (micro_df.iloc[[6]].participant_id).values[0],
            (micro_df.iloc[[6]].champion_name).values[0],
            (micro_df.iloc[[6]].role).values[0],
            (micro_df.iloc[[6]].level).values[0],
            (micro_df.iloc[[6]].kills).values[0],
            (micro_df.iloc[[6]].deaths).values[0],
            (micro_df.iloc[[6]].assists).values[0],
            (micro_df.iloc[[6]].total_gold_earned).values[0],
            (micro_df.iloc[[6]].creep_score).values[0],
            (micro_df.iloc[[6]].kill_participation).values[0],
            (micro_df.iloc[[6]].champion_damage_share).values[0],
            (micro_df.iloc[[6]].wards_placed).values[0],
            (micro_df.iloc[[6]].wards_destroyed).values[0],           
            (micro_df.iloc[[7]].participant_id).values[0],
            (micro_df.iloc[[7]].champion_name).values[0],
            (micro_df.iloc[[7]].role).values[0],
            (micro_df.iloc[[7]].level).values[0],
            (micro_df.iloc[[7]].kills).values[0],
            (micro_df.iloc[[7]].deaths).values[0],
            (micro_df.iloc[[7]].assists).values[0],
            (micro_df.iloc[[7]].total_gold_earned).values[0],
            (micro_df.iloc[[7]].creep_score).values[0],
            (micro_df.iloc[[7]].kill_participation).values[0],
            (micro_df.iloc[[7]].champion_damage_share).values[0],
            (micro_df.iloc[[7]].wards_placed).values[0],
            (micro_df.iloc[[7]].wards_destroyed).values[0],
            (micro_df.iloc[[8]].participant_id).values[0],
            (micro_df.iloc[[8]].champion_name).values[0],
            (micro_df.iloc[[8]].role).values[0],
            (micro_df.iloc[[8]].level).values[0],
            (micro_df.iloc[[8]].kills).values[0],
            (micro_df.iloc[[8]].deaths).values[0],
            (micro_df.iloc[[8]].assists).values[0],
            (micro_df.iloc[[8]].total_gold_earned).values[0],
            (micro_df.iloc[[8]].creep_score).values[0],
            (micro_df.iloc[[8]].kill_participation).values[0],
            (micro_df.iloc[[8]].champion_damage_share).values[0],
            (micro_df.iloc[[8]].wards_placed).values[0],
            (micro_df.iloc[[8]].wards_destroyed).values[0],
            (micro_df.iloc[[9]].participant_id).values[0],
            (micro_df.iloc[[9]].champion_name).values[0],
            (micro_df.iloc[[9]].role).values[0],
            (micro_df.iloc[[9]].level).values[0],
            (micro_df.iloc[[9]].kills).values[0],
            (micro_df.iloc[[9]].deaths).values[0],
            (micro_df.iloc[[9]].assists).values[0],
            (micro_df.iloc[[9]].total_gold_earned).values[0],
            (micro_df.iloc[[9]].creep_score).values[0],
            (micro_df.iloc[[9]].kill_participation).values[0],
            (micro_df.iloc[[9]].champion_damage_share).values[0],
            (micro_df.iloc[[9]].wards_placed).values[0],
            (micro_df.iloc[[9]].wards_destroyed).values[0],
            game_winner]
    print(data)
    df_5_minutes = df_5_minutes.append(pd.Series(data, index=name_of_columns), ignore_index=True)


[104727755403883497, '10.16.330.9186', 103877554248683116, 103935523328473675, 2500, 0, 0, 0, 0, 2500, 0, 0, 0, 0, '2020-08-25 16:05:00.000000', 1, 'blue', 'red', 2, 4, 4, 2, 1, 0, 0, 1, 'Ornn', 'top', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 2, 'Hecarim', 'jungle', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 3, 'Zoe', 'mid', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 4, 'Ezreal', 'bottom', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 5, 'Yuumi', 'support', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 6, 'Renekton', 'top', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 7, 'Nidalee', 'jungle', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 8, 'Orianna', 'mid', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 9, 'Ashe', 'bottom', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 10, 'Nautilus', 'support', 1, 0, 0, 0, 500, 0, 0.0, 0.0, 0, 0, 1]
[104727755403883501, '10.16.330.9186', 103877675241047720, 103935523328473675, 7387, 0, 0, 0, 1, 7084, 0, 0, 0, 1, '2020-08-27 16:57:00.000000', 1, 'blue', 'red', 4, 4, 2, 2, 1, 0, 0, 1, 'Shen', 'top', 4, 0, 0, 0, 1345, 26, 0.0, 0.08

In [95]:
print(df_5_minutes.head())

    game_external_ref   patch_version  ... wards_destroyed_player_10 game_winner
0  104727755403883497  10.16.330.9186  ...                         0           1
1  104727755403883501  10.16.330.9186  ...                         0           0
2  104727755403949039  10.16.330.9186  ...                         0           1
3  104727755403949041  10.16.330.9186  ...                         0           0
4  104727755403949043  10.16.330.9186  ...                         0           0

[5 rows x 156 columns]
